In [1]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.core.rust.model import TriggerType, TimeInForce
from nautilus_trader.model import TradeTick
from nautilus_trader.model import InstrumentId

In [2]:
catalog = ParquetDataCatalog("/home/p/p-dev/nautilus/catalog")
instrument_id = InstrumentId.from_str("APTUSDT-PERP.BINANCE")

In [3]:
venue = BacktestVenueConfig(
    name="BINANCE",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USDT",
    starting_balances=["100 USDT"],
    default_leverage=5,
    leverages={"APTUSDT-PERP.BINANCE": 5}
)

In [4]:
data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=TradeTick,
    instrument_ids=[instrument_id],
    start_time="2024-12-10",
    end_time="2025-01-10"
)

In [5]:
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
              "instrument_id": instrument_id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="INFO"),
)

In [6]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.core.rust.common import LogColor
from nautilus_trader.model.events import PositionOpened, PositionChanged, OrderAccepted, OrderFilled
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig



class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 100
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        self.log.info("Initialization of MACDStrategy")
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        # Convenience
        self.position: Position | None = None

    
    def on_start(self):
        self.log.info("Start of MACDStrategy")
        
        self.instrument = self.cache.instrument(self.config.instrument_id)
        self.account = self.portfolio.account(self.instrument.venue)
        self.trade_size = self.instrument.make_qty(self.config.trade_size)

        self.subscribe_trade_ticks(instrument_id=self.config.instrument_id)


    def on_stop(self):
        self.log.info("Stop of MACDStrategy")
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_trade_ticks(instrument_id=self.config.instrument_id)
    

    def on_trade_tick(self, tick: TradeTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available
        self.macd.handle_trade_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry(tick)


    def check_for_entry(self, tick: TradeTick):
        quantity = self.instrument.make_qty(
            self.config.trade_size
        )
        
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=quantity,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT
            
            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=quantity,
            )
            self.position_open_price = tick.price
            self.submit_order(order)

    
    def on_position_opened(self, event: PositionOpened):
        self.position = self.cache.position(event.position_id)
        
    
    def on_position_changed(self, event: PositionChanged):
        self.log.info(f"After position changed to amount {self.position.quantity} balances locked: " + 
             f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.CYAN)

        self.log.info(f"Quote quantity approx. opened: {self.position.quantity * self.position.avg_px_open}", LogColor.CYAN)
     

    def on_dispose(self):
        pass  # Do nothing else

In [7]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
    chunk_size=1024*32
)

In [8]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

results: list[BacktestResult] = node.run()

2025-04-20T20:47:39.727540856Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-20T20:47:39.727572017Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-04-20T20:47:39.727573724Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-04-20T20:47:39.727574503Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2025. All rights reserved.
2025-04-20T20:47:39.727575248Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-20T20:47:39.727576190Z [INFO] BACKTESTER-001.BacktestEngine: 
2025-04-20T20:47:39.727576905Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-04-20T20:47:39.727577754Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-04-20T20:47:39.727579070Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2025-04-20T20:47

In [9]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

48529 USDT
2024-12-10T00:00:25.968000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] PositionOpened(instrument_id=APTUSDT-PERP.BINANCE, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=BINANCE-001, opening_order_id=O-20241210-000025-001-000-37, closing_order_id=None, entry=SELL, side=SHORT, signed_qty=-75.4, quantity=75.4, peak_qty=75.4, currency=USDT, avg_px_open=11.9298, avg_px_close=0.0, realized_return=0.00000, realized_pnl=-0.44975346 USDT, unrealized_pnl=0.00000000 USDT, ts_opened=1733788825968000000, ts_last=1733788825968000000, ts_closed=0, duration_ns=0)
2024-12-10T00:00:25.968000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=0.00000000 USDT
2024-12-10T00:00:25.968000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=77.80227951 USDT, locked=4.58748529 USDT, free=73.21479422 USDT)], margins=[MarginBalance(initial=0.00

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20241210-000005-001-000-1,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-001,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-002,MARKET,SELL,100.0,...,None,None,None,None,None,None,None,82a2ae04-ace0-4a8e-87f7-e242ab4569fd,2024-12-10 00:00:05.048000+00:00,2024-12-10 00:00:05.048000+00:00
O-20241210-000008-001-000-2,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-002,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-004,MARKET,BUY,100.0,...,None,None,None,None,None,None,None,e86adaf1-8e50-4e9f-9b7c-7f02f1ab4d92,2024-12-10 00:00:08.205000+00:00,2024-12-10 00:00:08.205000+00:00
O-20241210-000008-001-000-3,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-003,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-006,MARKET,BUY,100.0,...,None,None,None,None,None,None,None,24a2fccd-2262-4f50-ba2a-ef34036955aa,2024-12-10 00:00:08.218000+00:00,2024-12-10 00:00:08.218000+00:00
O-20241210-000009-001-000-4,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-004,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-008,MARKET,SELL,100.0,...,None,None,None,None,None,None,None,b6f77384-f6d2-4baf-aa5d-3d22f13498d4,2024-12-10 00:00:09.609000+00:00,2024-12-10 00:00:09.609000+00:00
O-20241210-000009-001-000-5,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-005,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-010,MARKET,SELL,100.0,...,None,None,None,None,None,None,None,efb2e51e-2cc3-47ed-a2c7-024e4e77bf7a,2024-12-10 00:00:09.609000+00:00,2024-12-10 00:00:09.609000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20241210-024315-001-000-238,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-238,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-473,MARKET,SELL,100.0,...,None,None,None,None,None,None,None,74b33872-bfe7-4acc-a5ba-85f3ea95219b,2024-12-10 02:43:15.255000+00:00,2024-12-10 02:43:15.255000+00:00
O-20241210-024315-001-000-239,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-239,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-475,MARKET,SELL,100.0,...,None,None,None,None,None,None,None,a103b0bb-dccb-4cac-952e-74ecbae15bba,2024-12-10 02:43:15.255000+00:00,2024-12-10 02:43:15.255000+00:00
O-20241210-024315-001-000-240,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-240,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-477,MARKET,BUY,100.0,...,None,None,None,None,None,None,None,2a0b0906-98fd-46f1-9092-b8f66f1d1e82,2024-12-10 02:43:15.263000+00:00,2024-12-10 02:43:15.263000+00:00


SELL 100.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-000031-001-000-45, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T00:00:31.160000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000031-001-000-45, account_id=BINANCE-001, ts_event=1733788831160000000)
2024-12-10T00:00:31.160000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=0.00000000 USDT
2024-12-10T00:00:31.160000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=74.25549632 USDT, locked=0.00000000 USDT, free=74.25549632 USDT)], margins=[], event_id=37eb7d6e-cfa1-47fe-a6eb-8215b917b5a6)
2024-12-10T00:00:31.160000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-0000

In [10]:
engine.trader.generate_positions_report()

rted=False, balances=[AccountBalance(total=70.94627860 USDT, locked=6.07471200 USDT, free=64.87156660 USDT)], margins=[MarginBalance(initial=0.00000000 USDT, maintenance=6.07471200 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=077263a1-562a-450e-845b-856db83ef8f4)
2024-12-10T00:00:35.756000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000035-001-000-50, venue_order_id=BINANCE-1-050, account_id=BINANCE-001, trade_id=BINANCE-1-099, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, order_side=BUY, order_type=MARKET, last_qty=4.2, last_px=11.90480 USDT, commission=0.02500008 USDT, liquidity_side=TAKER, ts_event=1733788835756000000)
2024-12-10T00:00:35.756000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE net_position=-95.8
2024-12-10T00:00:35.756000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_maint=5.81957410 USDT
2024-12-10T00:00:35.756000000Z [INFO] BACKTESTER-001.MACDSt

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
APTUSDT-PERP.BINANCE-MACDStrategy-000-1a9b87f0-c6a8-4eb3-9808-0a2fc38c8de6,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000005-001-000-1,O-20241210-000008-001-000-2,SELL,FLAT,0.0,100.0,1733788805048000000,2024-12-10 00:00:05.048000+00:00,1733788808205000000,2024-12-10 00:00:08.204999936+00:00,3.157000e+09,11.935601,11.928092,[1.19318464 USDT],0.00063,-0.44226464 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-54c40857-b06c-4093-8a95-07ea3423c475,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000008-001-000-3,O-20241210-000009-001-000-4,BUY,FLAT,0.0,100.0,1733788808218000000,2024-12-10 00:00:08.218000+00:00,1733788809609000000,2024-12-10 00:00:09.608999936+00:00,1.391000e+09,11.928199,11.929801,[1.19290001 USDT],0.00013,-1.03272001 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-43fa1208-568e-4ecb-9a31-ebc34b60370a,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000009-001-000-5,O-20241210-000010-001-000-6,SELL,FLAT,0.0,100.0,1733788809609000000,2024-12-10 00:00:09.609000+00:00,1733788810101000000,2024-12-10 00:00:10.100999936+00:00,4.920000e+08,11.929401,11.930667,[1.19300339 USDT],-0.00011,-1.31958339 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-cb5ae959-c95d-4574-a07e-99f985b452a5,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-7,O-20241210-000010-001-000-8,BUY,FLAT,0.0,100.0,1733788810101000000,2024-12-10 00:00:10.101000+00:00,1733788810181000000,2024-12-10 00:00:10.180999936+00:00,8.000000e+07,11.928269,11.919217,[1.19237427 USDT],-0.00076,-2.09755427 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-83f640c8-46f9-43c4-9a08-217a35dd3b0f,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-9,O-20241210-000010-001-000-10,SELL,FLAT,0.0,100.0,1733788810181000000,2024-12-10 00:00:10.181000+00:00,1733788810242000000,2024-12-10 00:00:10.241999872+00:00,6.100000e+07,11.919125,11.921684,[1.19204044 USDT],-0.00021,-1.44792044 USDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
APTUSDT-PERP.BINANCE-MACDStrategy-000-cb2e1b73-b786-4565-9a22-45147e35fbad,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-024314-001-000-233,O-20241210-024315-001-000-234,BUY,FLAT,0.0,100.0,1733798594915000000,2024-12-10 02:43:14.915000+00:00,1733798595190000000,2024-12-10 02:43:15.190000128+00:00,2.750000e+08,11.719099,11.717001,[1.17180501 USDT],-0.00018,-1.38162501 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-5035015a-45be-477a-8258-65580677bc01,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-024315-001-000-235,O-20241210-024315-001-000-236,SELL,FLAT,0.0,100.0,1733798595190000000,2024-12-10 02:43:15.190000+00:00,1733798595253000000,2024-12-10 02:43:15.252999936+00:00,6.300000e+07,11.716921,11.703998,[1.17104595 USDT],0.00110,0.12121405 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-288f5031-a489-455b-a9c0-9cebbc759c64,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-024315-001-000-237,O-20241210-024315-001-000-238,BUY,FLAT,0.0,100.0,1733798595253000000,2024-12-10 02:43:15.253000+00:00,1733798595255000000,2024-12-10 02:43:15.255000064+00:00,2.000000e+06,11.705292,11.694600,[1.16999461 USDT],-0.00091,-2.23909461 USDT


INANCE net_position=100.0
2024-12-10T00:00:39.347000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_maint=6.07512000 USDT
2024-12-10T00:00:39.347000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] PositionChanged(instrument_id=APTUSDT-PERP.BINANCE, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=BINANCE-001, opening_order_id=O-20241210-000039-001-000-55, closing_order_id=None, entry=BUY, side=LONG, signed_qty=100.0, quantity=100.0, peak_qty=100.0, currency=USDT, avg_px_open=11.911972099999998, avg_px_close=0.0, realized_return=0.00000, realized_pnl=-0.59559861 USDT, unrealized_pnl=0.00279000 USDT, ts_opened=1733788839347000000, ts_last=1733788839347000000, ts_closed=0, duration_ns=0)
2024-12-10T00:00:39.347000000Z [INFO] BACKTESTER-001.MACDStrategy: After position changed to amount 100.0 balances locked: 6.07512
2024-12-10T00:00:39.347000000Z [INFO] BACKTESTER-001.MACDStrategy: Quote quantity approx. opened: 1191.1972099999998
2024-12-10T00:00:43.452000

In [11]:
engine.trader.generate_account_report(Venue("BINANCE"))

P.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-000045-001-000-60, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T00:00:45.206000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000045-001-000-60, account_id=BINANCE-001, ts_event=1733788845206000000)
2024-12-10T00:00:45.206000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=0.00000000 USDT
2024-12-10T00:00:45.206000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=67.62518886 USDT, locked=6.07884300 USDT, free=61.54634586 USDT)], margins=[MarginBalance(initial=0.00000000 USDT, maintenance=6.07884300 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=330900b1-7377-473e-ba5c-118c95b1a4a7)
2024-12-10T00:00:45.206000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2024-12-10 00:00:04.998000+00:00,100.00000000,0E-8,100.00000000,USDT,BINANCE-001,MARGIN,USDT,[],True,{}
2024-12-10 00:00:05.048000+00:00,99.99403215,0E-8,99.99403215,USDT,BINANCE-001,MARGIN,USDT,[],False,{}
2024-12-10 00:00:05.048000+00:00,99.40321995,0.06087207,99.34234788,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:08.205000+00:00,99.41664335,6.08715600,93.32948735,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:08.205000+00:00,99.55773536,5.58800921,93.96972615,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
...,...,...,...,...,...,...,...,...,...,...
2024-12-10 02:43:15.255000+00:00,7.53178859,0.14314190,7.38864669,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 02:43:15.263000+00:00,7.45890449,5.96419500,1.49470949,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 02:43:15.263000+00:00,6.80825410,5.35584711,1.45240699,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 02:43:15.263000+00:00,6.54804257,0E-8,6.54804257,USDT,BINANCE-001,MARGIN,USDT,[],False,{}


ntBalance(total=65.89388351 USDT, locked=0.00000000 USDT, free=65.89388351 USDT)], margins=[], event_id=d8b08f41-d8ef-4713-9452-c702f609f311)
2024-12-10T00:00:46.384000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000046-001-000-63, venue_order_id=BINANCE-1-063, account_id=BINANCE-001, trade_id=BINANCE-1-125, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, order_side=BUY, order_type=MARKET, last_qty=55.0, last_px=11.91350 USDT, commission=0.32762125 USDT, liquidity_side=TAKER, ts_event=1733788846384000000)
2024-12-10T00:00:46.384000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE net_position=55.0
2024-12-10T00:00:46.384000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_maint=3.34173675 USDT
2024-12-10T00:00:46.384000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] PositionOpened(instrument_id=APTUSDT-PERP.BINANCE, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=